In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from io import StringIO

In [2]:
#id = Seriale
#nom = Nome
#long_parcours = TotLungLato (en m)
#dist_parcouru = TotDistPerc (en m)
#temps_total = TotTempPerc (en s)
#s*_topspeed = cInfoRaceSegment/TopSpeed (en Knt)
#s*_topvmg = cInfoRaceSegment/TopVMG (en Knt)
#s*_avgspeed = cInfoRaceSegment/AvgSpeed (en Knt)
#s*_tempsgauche = cInfoRaceSegment/CrtRaceSegSX (en %)
#s*_tempsdroite = cInfoRaceSegment/CrtRaceSegDX (en %)
#s*_tempssursegment = cInfoRaceSegment/TimeSecPercorsi (en s)
#s*_distancesursegment = cInfoRaceSegment/SegDistRealePercorsa (en m)
#s*_longueursegment = cInfoRaceSegment/LungLato (en m)
#s*_rankentersegment = cInfoRaceSegment/SegEnteredRank

# M7, M8, M9, M10, M11, M12,  Départ – 1 – 4 – 1 – 2 – 3 - Arrivée
# M13  Départ – 1 – 2 – 3 – 2 – 3 - Arrivée
#https://app.metasail.it/(S(usryiluj43x0xh10ztf4whfp))/MetaSailWS.asmx/getStatistiche?idgara=xxx

manche = 'M9'

# remove namespace
src = open('./' + manche + '.xml', 'r').read()
tree = ET.iterparse(StringIO(src))
for _, el in tree:
    prefix, has_namespace, postfix = el.tag.partition('}')
    if has_namespace:
        el.tag = postfix

root = tree.root

rows = []
rows_repeat = []
row_index = 0
for node in root: 
    curr_segment = 1
    for segment in node.find('lstSegments').iter('cInfoRaceSegment'):
        row = {
            'id': node.find('Seriale').text if node is not None else None,
            'nom': node.find('Nome').text if node is not None else None,
            'long_parcours': int(node.find('TotLungLato').text) if node is not None else None,
            'dist_parcouru': int(node.find('TotDistPerc').text) if node is not None else None,
            'temps_total': int(node.find('TotTempPerc').text) if node is not None else None,
        }

        row['s_topspeed'] = float(segment.find('TopSpeed').text)
        row['s_topvmg'] = float(segment.find('TopVMG').text)
        row['s_avgspeed'] = float(segment.find('AvgSpeed').text)
        row['s_tempsgauche'] = int(segment.find('CrtRaceSegSX').text)
        row['s_tempsdroite'] = int(segment.find('CrtRaceSegDX').text)
        row['s_tempssursegment'] = int(segment.find('TimeSecPercorsi').text)
        row['s_distancesursegment'] = int(segment.find('SegDistRealePercorsa').text)
        row['s_longueursegment'] = int(segment.find('LungLato').text)
        row['s_rankentersegment'] = int(segment.find('SegEnteredRank').text)
        row['s_leg'] = curr_segment
        rows_repeat.append(row)
        curr_segment += 1

df = pd.DataFrame(rows_repeat, columns=[k for k in rows_repeat[0]])
df.dtypes


id                       object
nom                      object
long_parcours             int64
dist_parcouru             int64
temps_total               int64
s_topspeed              float64
s_topvmg                float64
s_avgspeed              float64
s_tempsgauche             int64
s_tempsdroite             int64
s_tempssursegment         int64
s_distancesursegment      int64
s_longueursegment         int64
s_rankentersegment        int64
s_leg                     int64
dtype: object

In [3]:
#Noel A0708
#Briez A0183
#Ganivez A0535
#Swinyard A0354
#vonck A0309

selection = df[df.id.isin(['A0535', 'A0708', 'A0183', 'A0354', 'A0309'])]

stat_selection = '_avgspeed'#'_distancesursegment'
stat_compare = '_avgspeed'#'_longueursegment'
selection

,id,nom,long_parcours,dist_parcouru,temps_total,s_topspeed,s_topvmg,s_avgspeed,s_tempsgauche,s_tempsdroite,s_tempssursegment,s_distancesursegment,s_longueursegment,s_rankentersegment,s_leg
60,A0183,Briez T.,3503,3974,1953,5.0,3.60,3.25,52,48,564,944,733,1,1
61,A0183,Briez T.,3503,3974,1953,6.8,6.82,5.09,0,100,264,691,668,18,2
62,A0183,Briez T.,3503,3974,1953,5.1,3.34,3.14,41,59,534,862,668,10,3
63,A0183,Briez T.,3503,3974,1953,6.8,3.29,4.95,0,100,258,658,632,19,4
64,A0183,Briez T.,3503,3974,1953,6.0,4.96,4.59,91,9,282,665,655,15,5
65,A0183,Briez T.,3503,3974,1953,6.9,5.97,5.87,100,0,51,154,147,14,6
174,A0535,Ganivet C.,3504,3992,1803,5.1,3.70,3.59,52,48,504,931,733,4,1
175,A0535,Ganivet C.,3504,3992,1803,6.6,6.40,5.09,0,100,267,699,668,1,2
176,A0535,Ganivet C.,3504,3992,1803,5.4,4.10,3.57,93,7,486,892,668,1,3
177,A0535,Ganivet C.,3504,3992,1803,8.0,3.73,6.18,0,100,201,639,632,1,4


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def add_graph(index, fig, row, col):
    m = df[df.s_leg == int(index)]
    mean = np.nanmean(m['s' + stat_compare])
    rows = selection[selection.s_leg == int(index)]

    t = go.Bar(
        x=rows.nom, 
        y=rows['s' + stat_selection],
        name='v moy : ' + str(round(mean, 2)) + 'knt',
        text=rows['s_rankentersegment']
    )
    
    fig.append_trace(t, row, col)
    fig.add_hline(y=mean, line_color='red', row=row, col=col)

fig_rows = 3
fig_cols = 2

fig = make_subplots(rows=fig_rows, cols=fig_cols, start_cell='top-left', subplot_titles=("Départ -> 1 (près)", "Leg 2", "Leg 3", "Leg 4", "Leg 5", "3 -> arrivée (travers)"))
index = 1
for row in range(fig_rows):
    for col in range(fig_cols):
        add_graph(str(index), fig, row + 1, col + 1)
        index += 1


fig.update_layout(title_text= manche + ' :: Vitesse moyenne / flotte par segment', height=900)
fig.update_yaxes(type='log')
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
df_leg = df[df.s_leg == 3]
nb=10
sel = df_leg[['nom', 's_distancesursegment', 's_avgspeed', 's_tempsgauche', 's_tempsdroite', 's_rankentersegment']]


temps_gauche = sel.nlargest(nb, ['s_tempsgauche']).sort_values('s_rankentersegment')
#print(temps_gauche)
d = temps_gauche.describe()
print('vmoy gauche\t', d.loc[['mean']].s_avgspeed.values[0])
print('dmoy gauche\t', d.loc[['mean']].s_distancesursegment.values[0])
print('temps moy gauche\t', d.loc[['mean']].s_tempsgauche.values[0])
print('median gauche\t', d.loc[['50%']].s_rankentersegment.values[0])
print('q .25 gauche\t', d.loc[['25%']].s_rankentersegment.values[0])
print('q .75 gauche\t', d.loc[['75%']].s_rankentersegment.values[0])
print('sum rank gauche\t', temps_gauche.s_rankentersegment.sum())

print()
temps_droite = sel.nlargest(nb, ['s_tempsdroite']).sort_values('s_rankentersegment')
#print(temps_droite)
d = temps_droite.describe()
print('vmoy droite\t', d.loc[['mean']].s_avgspeed.values[0])
print('temps moy droite\t', d.loc[['mean']].s_tempsdroite.values[0])
print('dmoy droite\t', d.loc[['mean']].s_distancesursegment.values[0])
print('median droite\t', d.loc[['50%']].s_rankentersegment.values[0])
print('q .25 droite\t', d.loc[['25%']].s_rankentersegment.values[0])
print('q .75 droite\t', d.loc[['75%']].s_rankentersegment.values[0])
print('sum rank droite\t', temps_droite.s_rankentersegment.sum())

NameError: name 'df' is not defined

In [ ]:
import math
df_leg = df[df.s_leg == 3]

sel = df_leg[['nom', 's_distancesursegment', 's_avgspeed', 's_tempsgauche', 's_tempsdroite', 's_rankentersegment']]

for nb in range(5, math.floor(len(sel) / 2)):
  temps_gauche = sel.nlargest(nb, ['s_tempsgauche'])
  d = temps_gauche.describe()
  print('vmoy gauche\t', d.loc[['mean']].s_avgspeed.values[0])
  print('dmoy gauche\t', d.loc[['mean']].s_distancesursegment.values[0])
  print('temps moy gauche\t', d.loc[['mean']].s_tempsgauche.values[0])
  print('median gauche\t', d.loc[['50%']].s_rankentersegment.values[0])
  print('q .25 gauche\t', d.loc[['25%']].s_rankentersegment.values[0])
  print('q .75 gauche\t', d.loc[['75%']].s_rankentersegment.values[0])
  print('sum rank gauche\t', temps_gauche.s_rankentersegment.sum())

In [5]:
import numpy as np
 
 
df= pd.DataFrame({'number': np.random.randint(1, 100, 10)})
df['bins'] = pd.cut(x=df['number'], bins=[1, 20, 40, 60,
                                          80, 100])
print(df)
 
# We can check the frequency of each bin
print(df['bins'].unique())

   number       bins
0      12    (1, 20]
1      50   (40, 60]
2      68   (60, 80]
3      17    (1, 20]
4      65   (60, 80]
5      27   (20, 40]
6      13    (1, 20]
7      97  (80, 100]
8      49   (40, 60]
9       2    (1, 20]
[(1, 20], (40, 60], (60, 80], (20, 40], (80, 100]]
Categories (5, interval[int64, right]): [(1, 20] < (20, 40] < (40, 60] < (60, 80] < (80, 100]]
